In [ ]:
colab=1
if colab:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/Projects/NepSense/Prashant/Model Training


In [ ]:
# !pip install pandas==2.2.2
# !pip install numpy==1.26.4
# !pip install nltk=3.8.1
# !pip install pycontractions
# !pip install scikit-learn
# !pip install tqdm
# !pip install tensorflow=2.16.1
# !pip install numpy
# !pip install seaborn
# !pip install matplotlib
# !pip install langdetect
# !pip install keras==3.3.3

In [ ]:
import tensorflow as tf

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Explicitly allow memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


In [ ]:
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
import pandas as pd
import numpy as np
import seaborn as sn
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense,Flatten, GlobalMaxPooling1D,GlobalAveragePooling1D, Embedding, Conv1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from keras.layers import Activation, Dropout, Dense
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download("stopwords")
# wordnet is foro lemmanation and stemmation
nltk.download("wordnet")
# punkt is for sent_tokenizer
nltk.download("punkt")

In [ ]:
import pandas as pd
dataset="./Model Training Datas/embeddings.csv"
df = pd.read_csv(dataset)
# Assuming df is your DataFrame
# Select rows with label 0 and 2
df = df[df['Label'].isin([0, 1])]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'df' is your DataFrame with the 'SentimentScore' column
sns.set(style="darkgrid")

# Create a countplot
ax = sns.countplot(x='Label', data=df)

# Add count annotations on top of the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

# Set labels and title
ax.set_xlabel("Sentiment Score")
ax.set_ylabel("Count")
ax.set_title("Sentiment Score Count")

# Show the plot
plt.show()

In [ ]:
df

In [ ]:
X=df["preprocessing_text"]
Y=df["Label"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20,stratify=Y, random_state=42)
# The train set will be used to train our deep learning models
# while test set will be used to evaluate how well our model performs

In [ ]:
unique_labels = set(y_train)
print(unique_labels)


In [ ]:
# print(y_train.count())
# print(y_test.count())
# X_train.head()
# Create a countplot
# Count occurrences of each value
value_counts = np.bincount(y_train)

# Print the counts
print("Counts of 1:", value_counts[0])
print("Counts of 2:", value_counts[1])


In [ ]:
type(X_train)

In [ ]:

# Embedding layer expects the words to be in numeric form
# Using Tokenizer function from keras.preprocessing.text library
# Method fit_on_text trains the tokenizer
# Method texts_to_sequences converts sentences to their numeric form
#Must run above shell
word_tokenizer =  Tokenizer(num_words=90000,
                      lower=True,
                      split=' ',
                      char_level=False,
                      oov_token='<UNK>',
                      document_count=0)

word_tokenizer.fit_on_texts(X_train)

wordindex=word_tokenizer.word_index
X_train = word_tokenizer.texts_to_sequences(X_train)
X_test = word_tokenizer.texts_to_sequences(X_test)


In [ ]:
# Adding 1 to store dimensions for words for which no pretrained word embeddings exist

vocab_length = len(wordindex) + 1

vocab_length

In [ ]:
# Padding all reviews to fixed length 100
max_len=500
X_train = pad_sequences(X_train, padding='post',maxlen=max_len)
X_test = pad_sequences(X_test, padding='post',maxlen=max_len)

In [ ]:
#print(test_padding[0])
#train_padding[0]
X_train.shape
y_train.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Create an instance of OneHotEncoder
encoder = OneHotEncoder()

# Convert pandas Series to NumPy array
y_train_array = np.array(y_train).reshape(-1, 1)
y_test_array = np.array(y_test).reshape(-1, 1)

# Fit and transform on training data
y_train = encoder.fit_transform(y_train_array)

# Transform only on test data
y_test = encoder.transform(y_test_array)

# Check the shape of the transformed arrays
print("Shape of y_train_encoded:", y_train.shape)
print("Shape of y_test_encoded:", y_test.shape)


In [ ]:
# # Neural Network architecture
from keras.models import Sequential
from keras.layers import Embedding, Dropout, Bidirectional, LSTM, Dense, Input
from keras.constraints import max_norm
from keras.regularizers import l2
from tensorflow.keras.utils import plot_model
from keras.optimizers import Adam

# Define the regularizer
regularise = l2(0.0001)

# Create the Sequential model
binomial_lstm_offensive_model = Sequential()
binomial_lstm_offensive_model.add(Input(shape=(X_train.shape[1],)))
embedding_layer = Embedding(vocab_length, 256)
binomial_lstm_offensive_model.add(embedding_layer)
binomial_lstm_offensive_model.add(Dropout(0.7))
binomial_lstm_offensive_model.add(Bidirectional(LSTM(128, return_sequences=True)))
binomial_lstm_offensive_model.add(Bidirectional(LSTM(64, kernel_constraint=max_norm(3))))
binomial_lstm_offensive_model.add(Dense(64, activation='relu', kernel_regularizer=regularise))
binomial_lstm_offensive_model.add(Dropout(0.7))
binomial_lstm_offensive_model.add(Dense(2, activation='sigmoid'))
plot_model(binomial_lstm_offensive_model, to_file='model_diagram.png', show_shapes=True, show_layer_names=True)
optimizer = Adam(learning_rate=0.00006)
binomial_lstm_offensive_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
print(binomial_lstm_offensive_model.summary())


In [ ]:
plot_model(binomial_lstm_offensive_model, to_file='model_diagram.png', show_shapes=True, show_layer_names=True)

In [ ]:
X_train

In [ ]:
y_train=np.array(y_train.toarray())
X_train=np.array(X_train)
y_test=np.array(y_test.toarray())
X_test=np.array(X_test)

In [ ]:
from keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch < 4:
        return lr
    else:
        return lr * 0.1

lr_scheduler = LearningRateScheduler(scheduler)

In [ ]:
from keras.models import save_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import json
# Train the model
epochs =30
batch_size = 128
# modeltrain=lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False)
# lr_scheduler = LearningRateScheduler(scheduler)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0000001)

regularizemodel=binomial_lstm_offensive_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2,callbacks=[early_stopping,lr_scheduler])


# # Assuming you have a Keras model named 'model'
# lstm_model.save('/content/drive/My Drive/CommentSenseData/LSTMsentimentmodel.h5')


In [ ]:
import matplotlib.pyplot as plt
# Function to plot training and validation accuracy
def plot_accuracy(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Offensive Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

# Function to plot training and validation loss
def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Offensive Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

  # Plot training and validation accuracy
plot_accuracy(regularizemodel)

# Plot training and validation loss
plot_loss(regularizemodel)

In [ ]:
epochs = 5
batch_size = 128
score = binomial_lstm_offensive_model.evaluate(X_test,y_test,batch_size=batch_size)
print("Testing Accuracy(%): ", score[1]*100)

In [ ]:
y_predictions = binomial_lstm_offensive_model.predict(X_test)
y_pred_labels = np.array([ np.argmax(i) for i in y_predictions])
y_test_labels = np.array([ np.argmax(i) for i in y_test])


# print(y_predictions)
# print(y_pred_labels)
# print(y_test_labels)

In [ ]:
confusion = confusion_matrix(y_test_labels, y_pred_labels)
labels=['None','Offensive']
plt.figure(figsize=(4,4))
sn.heatmap(confusion,  xticklabels=labels, yticklabels=labels, fmt='d', annot=True, cmap=plt.cm.Blues)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:

print('\nClassification Report for Bidirectional LSTM\n')
print(classification_report(y_test_labels, y_pred_labels, target_names=['Class  None',	 'Class Offensive']))

In [ ]:
import pickle

binomial_lstm_offensive_model_name= 'Binomial_LSTM_Offensive'
binomial_lstm_offensive_model_path= f'/content/drive/MyDrive/Projects/NepSense/{binomial_lstm_offensive_model_name}.keras'
binomial_lstm_offensive_tokenizer_path= f'/content/drive/MyDrive/Projects/NepSense/{binomial_lstm_offensive_model_name}.pkl'

binomial_lstm_offensive_model.save(binomial_lstm_offensive_model_path)
# Save the tokenizer
with open(binomial_lstm_offensive_tokenizer_path, 'wb') as f:
    pickle.dump(word_tokenizer, f)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras import initializers

import pickle
from tensorflow.keras.models import load_model

# Load the model
binomial_lstm_offensive_model = load_model(binomial_lstm_offensive_model_path)

# Load the tokenizer
with open(binomial_lstm_offensive_tokenizer_path, 'rb') as f:
    binomial_lstm_offensive_tokenizer = pickle.load(f)

# Function to preprocess and predict the sentiment of user input
def predict_sentiment(input_text):
    # Preprocess the input text
    input_sequence = binomial_lstm_offensive_tokenizer.texts_to_sequences([input_text])
    input_padded = pad_sequences(input_sequence, maxlen=500, padding='post')

    # Predict the sentiment
    prediction = binomial_lstm_offensive_model.predict(input_padded)
    predicted_label = prediction.argmax(axis=-1)[0]

    # Map the predicted label to the sentiment class
    label_map = {0: 'Non-Offensive', 1: 'Offensive'}
    result = label_map[predicted_label]

    return result

# Ask the user for input and display the result
user_input = input("Enter a sentence to check for offensive: ")
sentiment_result = predict_sentiment(user_input)
print(f"The sentence is: {sentiment_result}")
